<div style="background-color: #ADD8E6; border: 1px solid gray; padding: 3px">
    This notebook consists of 2 agentic workflows:
        <h3>Data Generation Workflow</h3>
        <li><b>Data Augmentation</b>: Augments the provided image dataset.</li>
        <h3>Validation Workflow</h3>
        <li><b>Image Validator</b>: Identifies whether a valid driver's license exists in the given image.</li>
        <li><b>Data Extractor</b>: Extracts relevant metadata from the image.</li>
        <li><b>Application Validator</b>: Given the extracted metadata associated with the application, uses a set of predefined rules to validate the driver's license application.</li>
</div>

In [1]:
##############################################################################
# Imports
##############################################################################
import importlib
from crewai import Agent, Task, Crew, Process, LLM
from crewai.project import CrewBase, agent, crew, task
from crewai.agents.agent_builder.base_agent import BaseAgent
from typing import List, Optional
from pydantic import Field, BaseModel
from crewai_tools import SerperDevTool
from crewai.tools import tool
from crewai.flow.flow import Flow, listen, start
from crew import ValidateDriversLicense
import os
from urllib.parse import urlparse
from dotenv import load_dotenv
load_dotenv()
import base64
import json
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [2]:
# ##############################################################################
# # Structured Output
# ##############################################################################

# class DriversLicenseField(BaseModel):
    
#     field: str = Field(description="Name of field", default=None)
    
#     error_reason: str = Field(description="Reason for invalid or missing field", default=None)

# class DriversLicenseMetadata(BaseModel):
    
#     name: DriversLicenseField = Field(description="Name of driver's license owner")
    
#     date_of_birth: DriversLicenseField = Field(description="Date of birth of driver's license owner")
    
#     expiration_date: DriversLicenseField = Field(description="Expiration date of driver's license")
    
#     state_issued: DriversLicenseField = Field(description="State where the license was issued")

In [3]:
def get_base64encoded_image(image_path):
    """Generates a base64-encoded image from the give image path."""
    
    try:
        with open(image_path, "rb") as image_file:
            
            binary_image_data = image_file.read()
            
            base64_encoded_data = base64.b64encode(binary_image_data)
            
            base64_string = base64_encoded_data.decode('utf-8')
            
            return base64_string
            
    except Exception as e:
        print(f"Error occurred while generating base64-encoded image for {image_path}: {e}")
        
        return None

In [4]:
##############################################################################
# Flows
##############################################################################

class DriversLicenseValidationFlow(Flow):
    """Flow for Driver's License Validation."""

    @start()
    def get_image(self):
        
        print(f"Starting flow {self.state['id']} for {self.state["image_path"]}...")

        image_path = self.state["image_path"]

        image = get_base64encoded_image(image_path)

        self.state["image"] = image

    @listen(get_image)
    def validate_image(self):
        
        print("Validating image!...")

        image = self.state["image"]

        results = ValidateDriversLicense().crew().kickoff(inputs={"image": image})

        return results

### Execute Code Translation Flow
Execute the flow!

In [5]:
##############################################################################
# Execute the Flow
##############################################################################
paths = [
    ("data/DENVER-25CAP-00000-04SUF-ID.jpeg", "data/DENVER-25CAP-00000-04SUF-ID.json")
]

flow = DriversLicenseValidationFlow()

flow.plot("DriversLicenseValidationFlow")

for image_path, application_path in paths:

    results = flow.kickoff(inputs={"image_path": image_path,
                                  "application": application_path})

    if results.json_dict:
        
        print(json.dumps(results.json_dict, indent=2))

    if results.pydantic:
        
        print(results.pydantic.model_dump_json(indent=2))

    else:

        print(results.raw)

    print("Driver's License Validation flow complete.")

╭──────────────────────────────────────────────── Flow Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name: DriversLicenseValidationFlow                                                                             │
│  ID: 1e52d358-dc50-428d-b573-27cca2433558                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

 Flow started with ID: 1e52d358-dc50-428d-b573-27cca2433558

/opt/miniconda3/lib/python3.12/site-packages/rich/live.py:256: UserWarning: install "ipywidgets" for Jupyter 
support
  warnings.warn('install "ipywidgets" for Jupyter support')

Starting flow 1e52d358-dc50-428d-b573-27cca2433558 for data/DENVER-25CAP-00000-04SUF-ID.jpeg...

Validating image!...

Plot saved as DriversLicenseValidationFlow.html


/opt/miniconda3/lib/python3.12/site-packages/rich/live.py:256: UserWarning: install "ipywidgets" for Jupyter 
support
  warnings.warn('install "ipywidgets" for Jupyter support')

Error executing listener validate_image: litellm.BadRequestError: OpenAIException - This endpoint's maximum context
length is 131072 tokens. However, you requested about 211357 tokens (203165 of text input, 8192 in the output). 
Please reduce the length of either one, or use the "middle-out" transform to compress your prompt automatically.

BadRequestError: litellm.BadRequestError: OpenAIException - This endpoint's maximum context length is 131072 tokens. However, you requested about 211357 tokens (203165 of text input, 8192 in the output). Please reduce the length of either one, or use the "middle-out" transform to compress your prompt automatically.